# Amazon Bedrock AgentCore Gateway - 시맨틱 검색 튜토리얼

튜토리얼 세부 정보


| 정보                | 세부 사항                                                                        |
|:--------------------|:---------------------------------------------------------------------------------|
| 튜토리얼 유형        | 대화형                                                                           |
| 에이전트 유형        | 단일                                                                             |
| AgentCore 서비스    | AgentCore Gateway, AgentCore Identity                                            |
| 에이전트 프레임워크   | Strands Agents                                                                   |
| LLM 모델            | Anthropic Claude Sonnet 3.7                                                        |
| 튜토리얼 구성 요소    | Strands Agent에서 Lambda 기반 AgentCore Gateway 생성 및 사용                      |
| 튜토리얼 분야        | 교차 분야                                                                        |
| 예제 복잡성          | 쉬움                                                                             |
| 사용된 SDK          | Amazon BedrockAgentCore Python SDK 및 boto3  

### 튜토리얼 아키텍처
Amazon Bedrock AgentCore Gateway는 에이전트와 상호 작용해야 하는 도구 및 리소스 간의 통합 연결을 제공합니다. Gateway는 이 연결 계층에서 여러 역할을 수행합니다:

1. **보안 가드**: Gateway는 OAuth 권한 부여를 관리하여 유효한 사용자/에이전트만 도구/리소스에 액세스할 수 있도록 합니다.
2. **Translator**: Gateway는 모델 컨텍스트 프로토콜(MCP)과 같은 인기 있는 프로토콜을 사용하여 전송된 에이전트 요청을 API 요청 및 Lambda 호출로 변환합니다. 이는 개발자가 서버를 호스팅하거나 프로토콜 통합, 버전 지원, 버전 패치 관리 등을 수행할 필요가 없다는 것을 의미합니다.
3. **작곡가**: Gateway를 통해 개발자는 여러 API, 함수 및 도구를 에이전트가 사용할 수 있는 단일 MCP 엔드포인트로 원활하게 결합할 수 있습니다.
4. **Keychain**: Gateway는 적절한 도구와 함께 사용할 적절한 자격 증명을 주입하여 에이전트가 다양한 자격 증명 집합이 필요한 도구를 원활하게 활용할 수 있도록 보장합니다.
5. **Researcher**: Gateway를 통해 에이전트는 모든 도구를 검색하여 주어진 상황 또는 질문에 가장 적합한 도구만 찾을 수 있습니다. 이를 통해 에이전트는 소수의 도구가 아닌 수천 개의 도구를 활용할 수 있습니다. 또한 에이전트의 LLM 프롬프트에 제공해야 하는 도구 세트를 최소화하여 지연 시간과 비용을 절감합니다.
6. **Infrastructure Manager**: Gateway는 완전히 서버리스이며 내장된 관찰 기능과 감사 기능을 갖추고 있어 개발자가 에이전트를 통합하기 위해 추가 인프라를 관리할 필요가 없습니다.

![How does it work](images/gw-arch-overview.png)

### 튜토리얼 주요 기능

* AWS Lambda 기반 대상으로 Amazon Bedrock AgentCore Gateway 생성
* AgentCore Gateway 시맨틱 검색 사용
* AgentCore Gateway 검색이 지연 시간을 개선하는 방법을 보여주기 위해 Strands Agents 사용

## 전제 조건

이 튜토리얼을 실행하려면 다음이 필요합니다:
* Python 3.10+
* AWS 자격 증명
* Amazon Bedrock AgentCore SDK
* Strands Agents

## AgentCore Gateway는 많은 수의 도구를 가진 MCP 서버의 문제를 해결하는 데 도움이 됩니다
일반적인 기업 환경에서 에이전트 빌더는 수백 또는 수천 개의 MCP 도구를 가진 MCP 서버를 만나게 됩니다. 이러한 도구의 양은 AI 에이전트에게 도구 선택 정확도 저하, 비용 증가, 과도한 도구 메타데이터로 인한 높은 토큰 사용량으로 인한 지연 시간 증가 등의 문제를 야기합니다.
이는 에이전트를 타사 서비스(예: Zendesk, Salesforce, Slack, JIRA 등)나 기존 기업 REST 서비스에 연결할 때 발생할 수 있습니다. 
AgentCore Gateway는 도구 전반에 걸친 내장 시맨틱 검색을 제공하여 에이전트의 지연 시간, 비용 및 정확도를 개선하면서도 에이전트가 필요한 도구를 제공합니다. 
사용 사례, LLM 모델 및 에이전트 프레임워크에 따라 일반적인 MCP 서버의 수백 개 도구 전체 세트를 제공하는 것보다 관련 도구에 에이전트를 집중시킴으로써 최대 3배 더 나은 지연 시간을 볼 수 있습니다.

![How does it work](images/gateway_tool_search.png)

## 이 노트북에서 배울 내용
이 노트북에서는 AgentCore Gateway 검색에 대한 튜토리얼을 제공합니다. 이 단계별 튜토리얼을 마치면 다음을 이해하게 됩니다:

- AgentCore Gateway의 내장 검색 도구를 사용하여 관련 도구를 빠르게 찾는 방법 
- 지연 시간 개선 및 비용 절감을 위해 도구 검색 결과를 Strands Agents에 통합하는 방법

## 노트북 구조 개요
노트북은 다음 섹션으로 구성되어 있습니다:

1. AgentCore Gateway 검색의 기본 사항 이해
2. 노트북 환경 준비
3. 수백 개의 도구를 가진 Gateway 설정
4. Gateway에서 도구 검색
5. 많은 도구를 가진 MCP 서버와 함께 Strands Agents 사용
6. Strands Agent에 도구 검색 결과 추가
7. 도구 검색을 사용하여 3배 지연 시간 개선 보여주기

# AgentCore Gateway 검색의 기본 사항 이해

AgentCore Gateway를 생성할 때 검색을 활성화하려는 옵션을 선택할 수 있습니다.
검색이 활성화된 Gateway의 경우 세 가지 일이 발생합니다:

1. **벡터 저장소가 생성됩니다**. Gateway 서비스는 새 Gateway를 위해 서버리스 완전 관리형 벡터 저장소를 자동으로 생성합니다. 이를 통해 Gateway 도구 전반에 걸친 완전한 시맨틱 검색이 가능합니다. 
3. **벡터 저장소가 채워집니다**. Gateway에 Gateway 대상을 추가하면 서비스는 새 대상의 도구를 기반으로 벡터 저장소를 채우기 위해 백그라운드에서 임베딩을 자동으로 사용합니다. 도구 메타데이터는 도구의 JSON 정의 또는 REST 서비스 대상의 OpenAPI 스키마 사양에서 가져옵니다.
2. **검색 도구(MCP 기반)가 제공됩니다**. 모든 사용자 정의 도구(AWS Lambda 대상 또는 REST 서비스에서) 외에도 Gateway는 시맨틱 검색을 제공하는 하나의 추가 MCP 도구를 얻습니다. 이름은 `x-amz-bedrock-agentcore-search`입니다. 접두사는 사용자 정의 도구와 이름 충돌이 없도록 보장합니다. 향후에도 이와 같은 도구를 더 추가할 수 있습니다. 검색 도구에는 `query`라는 단일 인수가 있습니다. 검색 도구가 호출되면 Gateway 서비스는 해당 쿼리를 사용하여 시맨틱 검색을 수행하고 사용 가능한 도구 메타데이터(이름, 설명, 입력 및 출력 스키마)와 일치시켜 관련성 내림차순으로 가장 관련성이 높은 도구를 반환합니다.

# 노트북 환경 준비

In [ ]:
!pip install --force-reinstall -U -r requirements.txt --quiet

필요한 모든 Python 라이브러리를 가져오고 환경 변수를 로드합니다

In [ ]:
from strands import Agent
from strands.models import BedrockModel
from strands.handlers import null_callback_handler

from strands.tools.mcp.mcp_client import MCPClient, MCPAgentTool

from mcp.client.streamable_http import streamablehttp_client
from mcp.types import Tool as MCPTool

import logging
import time
import json
import boto3
import requests
import utils

GATEWAY_NAME = "gateway-search-tutorial"

로거 설정

In [ ]:
# Configure the root strands logger
logging.getLogger("strands").setLevel(logging.ERROR)  # INFO) #DEBUG) #

# Add a handler to see the logs
logging.basicConfig(
    format="%(levelname)s | %(name)s | %(message)s", handlers=[logging.StreamHandler()]
)

boto3 버전 확인

In [ ]:
boto3.__version__

AgentCore 제어 평면 API용 boto3 클라이언트를 가져옵니다.

In [ ]:
session = boto3.Session()
agentcore_client = session.client(
    "bedrock-agentcore-control",
)

# 수백 개의 도구를 가진 Gateway 설정

AgentCore Gateway는 기존 API의 선별된 세트를 에이전트용 MCP 도구로 노출하는 안전하고 확장 가능한 방법을 제공합니다. 프로덕션 환경에서는 CloudFormation, CDK 또는 Terraform과 같은 도구를 사용하여 코드형 인프라로 Gateway 리소스를 생성합니다. 이 튜토리얼에서는 리소스와 API를 더 효과적으로 이해할 수 있도록 boto3 제어 평면 API를 직접 사용합니다.
이를 통해 자신만의 게이트웨이를 구축하고 사용하며 더 강력하고 안전한 에이전트를 만드는 데 더 쉽게 시작할 수 있습니다.

높은 수준에서 Gateway 설정 단계는 다음과 같습니다:

1. 인바운드(에이전트가 Gateway 호출) 및 아웃바운드(Gateway가 도구 호출) 보안을 위해 사용하는 ID 공급자 및 자격 증명 공급자를 정의합니다.
2. `create_gateway`를 사용하여 Gateway를 생성합니다.
3. `create_gateway_target`을 사용하여 Gateway 대상을 추가하여 AWS Lambda 또는 기존 RESTful 서비스에서 구현될 MCP 도구를 노출합니다.

이 튜토리얼에서는 Amazon Cognito를 ID 공급자(IdP)로, AWS Lambda 함수를 대상으로, AWS IAM을 아웃바운드 인증으로 사용합니다. 이 튜토리얼에서 설명하는 동일한 개념은 다른 IdP나 다른 대상 유형을 사용할 때도 여전히 적용됩니다.

### Amazon Cognito 리소스 생성

이 튜토리얼에서는 다음 리소스를 이미 생성했고 해당 환경 변수를 설정했다고 가정합니다:

- AWS Lambda 실행을 위한 IAM 역할 (`gateway_lambda_iam_role`)
- 간단한 수학 도구용 AWS Lambda 함수 (`calc_lambda_arn`)
- 레스토랑 예약 도구용 AWS Lambda 함수 (`restaurant_lambda_arn`)
- 클라이언트 ID (`cognito_client_id`)와 검색 URL (`cognito_discovery_url`)을 제공하는 Amazon Cognito 사용자 풀

레스토랑 API의 JSON 도구 메타데이터를 살펴보겠습니다. 기존 REST 서비스와 통합하는 경우 API 사양은 대신 OpenAPI 스키마를 사용하여 제공됩니다.

In [ ]:
with open("./restaurant/restaurant-api.json") as f:
    data = json.load(f)
print(json.dumps(data, indent=4))

다음은 간단한 계산기 API입니다.

In [ ]:
with open("./calc/calc-api.json") as f:
    data = json.load(f)[0:3]
print(json.dumps(data, indent=4))

다음은 계산기 도구용 AWS Lambda 함수 구현입니다.

In [ ]:
from IPython.display import display, Code

with open("./calc/lambda_function_code.py", "r") as f:
    code_content = f.read()
display(Code(code_content, language="python"))

In [ ]:
with open("./restaurant/lambda_function_code.py", "r") as f:
    code_content = f.read()
display(Code(code_content, language="python"))

In [ ]:
#### Create a sample AWS Lambda function that you want to convert into MCP tools
calc_lambda_resp = utils.create_gateway_lambda(
    "calc/lambda_function_code.zip", lambda_function_name="calc_lambda_gateway"
)

if calc_lambda_resp is not None:
    if calc_lambda_resp["exit_code"] == 0:
        print(
            "Lambda function created with ARN: ",
            calc_lambda_resp["lambda_function_arn"],
        )
    else:
        print(
            "Lambda function creation failed with message: ",
            calc_lambda_resp["lambda_function_arn"],
        )

In [ ]:
calc_lambda_resp["lambda_function_arn"]

In [ ]:
#### Create a sample AWS Lambda function that you want to convert into MCP tools
restaurant_lambda_resp = utils.create_gateway_lambda(
    "restaurant/lambda_function_code.zip",
    lambda_function_name="restaurant_lambda_gateway",
)

if restaurant_lambda_resp is not None:
    if restaurant_lambda_resp["exit_code"] == 0:
        print(
            "Lambda function created with ARN: ",
            restaurant_lambda_resp["lambda_function_arn"],
        )
    else:
        print(
            "Lambda function creation failed with message: ",
            restaurant_lambda_resp["lambda_function_arn"],
        )

In [ ]:
restaurant_lambda_resp["lambda_function_arn"]

In [ ]:
cognito_response = utils.setup_cognito_user_pool()

In [ ]:
bearer_token = utils.get_bearer_token(
    client_id=cognito_response["client_id"],
    username="testuser",
    password="MyPassword123!",
)

In [ ]:
gateway_role_arn = utils.create_gateway_iam_role(
    lambda_arns=[
        calc_lambda_resp["lambda_function_arn"],
        restaurant_lambda_resp["lambda_function_arn"],
    ]
)

#### 제어 평면 API 사용을 위한 몇 가지 도우미 함수를 생성해보겠습니다

In [ ]:
def read_apispec(json_file_path):
    try:
        # read json file and return contents as string
        with open(json_file_path, "r") as file:
            # Parse JSON to Python object
            api_spec = json.load(file)
            return api_spec

    except FileNotFoundError:
        return f"Error: File {json_file_path} not found"
    except Exception as e:
        return f"An unexpected error occurred: {str(e)}"


def list_gateways():
    response = agentcore_client.list_gateways()
    print(json.dumps(response, indent=2, default=str))
    return response

#### Gateway 생성 도우미 함수
이름과 설명이 주어진 AgentCore Gateway를 생성하는 도우미 함수입니다. Amazon Cognito를 IdP로 사용하고 이미 정의된 허용된 클라이언트 ID와 검색 URL을 환경 변수에서 가져옵니다.
또한 결과 Gateway에서 시맨틱 검색을 기본적으로 활성화하고 미리 정의된 IAM 역할을 사용합니다.

In [ ]:
def create_gateway(gateway_name, gateway_desc):
    # Use Cognito for Inbound OAuth to our Gateway
    auth_config = {
        "customJWTAuthorizer": {
            "allowedClients": [cognito_response["client_id"]],
            "discoveryUrl": cognito_response["discovery_url"],
        }
    }
    # Enable semantic search of tools
    search_config = {
        "mcp": {"searchType": "SEMANTIC", "supportedVersions": ["2025-03-26"]}
    }
    # Create the gateway
    response = agentcore_client.create_gateway(
        name=gateway_name,
        roleArn=gateway_role_arn,
        authorizerType="CUSTOM_JWT",
        description=gateway_desc,
        protocolType="MCP",
        authorizerConfiguration=auth_config,
        protocolConfiguration=search_config,
    )
    # print(json.dumps(response, indent=2, default=str))
    return response["gatewayId"]

#### Gateway 대상 생성 도우미 함수
이 함수는 기존 Gateway에 새 AWS Lambda 대상을 생성합니다.
게이트웨이 ID, 새 대상의 이름과 설명, 기존 AWS Lambda 함수의 ARN, 그리고 게이트웨이에서 노출하려는 도구의 인터페이스를 설명하는 JSON 스키마를 제공하기만 하면 됩니다.

In [ ]:
def create_gatewaytarget(gateway_id, target_name, target_descr, lambda_arn, api_spec):
    # Add a Lambda target to the gateway
    response = agentcore_client.create_gateway_target(
        gatewayIdentifier=gateway_id,
        name=target_name,
        description=target_descr,
        targetConfiguration={
            "mcp": {
                "lambda": {
                    "lambdaArn": lambda_arn,
                    "toolSchema": {"inlinePayload": api_spec},
                }
            }
        },
        # Use IAM as credential provider
        credentialProviderConfigurations=[
            {"credentialProviderType": "GATEWAY_IAM_ROLE"}
        ],
    )
    return response["targetId"]

### 첫 번째 AgentCore Gateway 생성
첫 번째 Gateway를 설정하기 전에 Gateway가 MCP 도구 사용을 위한 인바운드 요청과 Gateway에서 도구 및 리소스로의 아웃바운드 액세스 모두에 대해 보안을 제공하는 방법을 간단히 살펴보겠습니다.

![How does it work](images/gateway_secure_access.png)

이제 이름과 설명을 제공하여 이 튜토리얼용 게이트웨이를 생성해보겠습니다.

In [ ]:
print(f"Create gateway with name: {GATEWAY_NAME}")
gatewayId = create_gateway(
    gateway_name=GATEWAY_NAME, gateway_desc="AgentCore Gateway Tutorial"
)
print(f"Gateway created with id: {gatewayId}.")

### AgentCore Gateway 대상 추가
이 튜토리얼에서는 간단한 수학 계산을 수행하는 Lambda 함수 하나와 레스토랑 예약 생성을 시뮬레이션하는 또 다른 Lambda 함수 쌍을 이미 설치했다고 가정합니다. 이러한 각 함수에 대해 Gateway 대상을 추가할 것입니다.

이러한 대상을 추가한 후에는 AgentCore Gateway 검색의 힘을 보여주는 데 도움이 되는 더 높은 MCP 도구 수를 단순히 늘리기 위해 추가 대상을 추가할 것입니다.

게이트웨이가 생성되었으므로 Lambda 함수를 통해 레스토랑 예약을 만들기 위한 대상을 추가해보겠습니다.

In [ ]:
restaurant_api_spec = read_apispec("./restaurant/restaurant-api.json")
restaurant_lambda_arn = restaurant_lambda_resp["lambda_function_arn"]
print(f"Restaurant Lambda ARN: {restaurant_lambda_arn}")

restaurantTargetId = create_gatewaytarget(
    gateway_id=gatewayId,
    lambda_arn=restaurant_lambda_arn,
    target_name="FoodTools",
    target_descr="Restaurant Tools",
    api_spec=restaurant_api_spec,
)
print(f"RestaurantTarget created with id: {restaurantTargetId} on gateway: {gatewayId}")

여기서는 4개의 기본 도구(더하기, 빼기, 곱하기, 나누기)를 구현하는 Lambda와 투자 관리(거래, 신용 연구, 정량 분석, 포트폴리오 관리)를 위한 75개의 생성된 도구 정의 세트가 있는 두 번째 대상을 추가할 것입니다. 투자 관리 도구 정의는 실제로 Lambda 함수에서 구현되지 않습니다. 많은 양의 도구를 보여주기 위해서만 추가하고 있습니다.

In [ ]:
calc_api_spec = read_apispec("./calc/calc-api.json")
print(f"API spec for calc has {len(calc_api_spec)} functions\n")
calc_lambda_arn = calc_lambda_resp["lambda_function_arn"]
print(f"Calc Lambda ARN: {calc_lambda_arn}")

time.sleep(5)
calcTargetId = create_gatewaytarget(
    gateway_id=gatewayId,
    lambda_arn=calc_lambda_arn,
    target_name="CalcTools",
    target_descr="Calculation Tools",
    api_spec=calc_api_spec,
)
print(f"CalcTools Target created with id: {calcTargetId} on gateway: {gatewayId}")

게이트웨이 검색의 힘을 보여주기 위해 이제 Calculator 대상의 복사본을 몇 개 더 추가하여 300개 이상의 MCP 도구가 노출되도록 합니다.

In [ ]:
def add_more_tools(gatewayId):
    time.sleep(10)
    calcTargetId = create_gatewaytarget(
        gateway_id=gatewayId,
        lambda_arn=calc_lambda_arn,
        target_name="Calc2",
        target_descr="Calculation 2 Tools",
        api_spec=calc_api_spec,
    )
    print(f"Calc2 Target created with id: {calcTargetId} on gateway: {gatewayId}")
    time.sleep(10)
    calcTargetId = create_gatewaytarget(
        gateway_id=gatewayId,
        lambda_arn=calc_lambda_arn,
        target_name="Calc3",
        target_descr="Calculation 3 Tools",
        api_spec=calc_api_spec,
    )
    print(f"Calc3 Target created with id: {calcTargetId} on gateway: {gatewayId}")
    time.sleep(10)
    calcTargetId = create_gatewaytarget(
        gateway_id=gatewayId,
        lambda_arn=calc_lambda_arn,
        target_name="Calc4",
        target_descr="Calculation 4 Tools",
        api_spec=calc_api_spec,
    )
    print(f"Calc4 Target created with id: {calcTargetId} on gateway: {gatewayId}")

In [ ]:
add_more_tools(gatewayId=gatewayId)

In [ ]:
resp = agentcore_client.list_gateway_targets(gatewayIdentifier=gatewayId)
targets = resp["items"]
for target in resp["items"]:
    print(f"{target['name']} - {target['description']}")

# Gateway에서 도구 검색

### 검색하기 전에 MCP 목록 도구에 익숙해지기

주어진 Gateway ID에 대한 MCP 엔드포인트 URL을 검색하고 Gateway를 안전하게 사용하기 위한 JWT OAuth 액세스 토큰을 검색하는 몇 가지 유틸리티 함수를 정의해보겠습니다.

In [ ]:
def get_gateway_endpoint(gateway_id):
    response = agentcore_client.get_gateway(gatewayIdentifier=gateway_id)
    gateway_url = response["gatewayUrl"]
    return gateway_url

이제 Gateway가 생성되고 대상이 있으므로 해당 Gateway의 MCP URL을 가져와보겠습니다. Gateway ID를 기반으로 Gateway 제어 평면에서 엔드포인트 URL을 검색할 수 있습니다.

#### Gateway에 대해 MCP Inspector 사용하기

MCP 서버의 엔드포인트 URL과 JWT 베어러 토큰이 있으므로 MCP Inspector 도구로 MCP 서버를 탐색할 수 있습니다.
MCP Inspector는 모든 MCP 서버에 연결할 수 있는 오픈 소스 도구로,
제공되는 도구를 나열하고 사용하기 쉬운 도구 호출 경험을 제공합니다.

터미널 창에서 `npx @modelcontextprotocol/inspector`를 입력하여 MCP Inspector를 실행하기만 하면 됩니다. 그런 다음
Gateway 엔드포인트 URL과 JWT 토큰을 붙여넣어 연결합니다. 연결되면 도구 목록 및 도구 호출을 시도해보세요.

다음은 샘플 스크린샷입니다.

![MCP Inspector](images/mcp_inspector.png)

In [ ]:
gatewayEndpoint = get_gateway_endpoint(gateway_id=gatewayId)
print(f"Gateway Endpoint - MCP URL: {gatewayEndpoint}")

MCP 서버 보안은 OAuth를 기반으로 합니다. Gateway와 상호 작용하려면
IdP에서 JWT OAuth 액세스 토큰을 검색해야 합니다.

In [ ]:
jwtToken = utils.get_bearer_token(
    client_id=cognito_response["client_id"],
    username="testuser",
    password="MyPassword123!",
)
print(f"Bearer token: {jwtToken}")

In [ ]:
# !npx @modelcontextprotocol/inspector

#### jsonrpc를 사용하여 MCP 도구를 호출하거나 나열하는 도우미 함수 생성
jsonrpc를 사용하여 Gateway를 포함한 MCP 서버에서 노출하는 모든 MCP 도구를 호출하는
`invoke_gateway_tool`이라는 도우미 함수를 정의해보겠습니다. 엔드포인트 URL과 JWT 토큰이 주어지면
이 유틸리티를 사용하여 Gateway에 Gateway 대상을 추가할 때 AgentCore Gateway가
사용할 수 있게 한 모든 MCP 도구를 호출할 수 있습니다.

In [ ]:
def invoke_gateway_tool(gateway_endpoint, jwt_token, tool_params):
    # print(f"Invoking tool {tool_params['name']}")

    requestBody = {
        "jsonrpc": "2.0",
        "id": 2,
        "method": "tools/call",
        "params": tool_params,
    }
    response = requests.post(
        gateway_endpoint,
        json=requestBody,
        headers={
            "Authorization": f"Bearer {jwt_token}",
            "Content-Type": "application/json",
        },
    )

    return response.json()

다음은 MCP의 `tools/list` 메서드를 사용하여 게이트웨이에서 사용 가능한 MCP 도구를 나열하는 또 다른 유틸리티 함수입니다. 게이트웨이 ID와 JWT 토큰을 받으면 해당 게이트웨이에서 전체 도구 세트를 검색하여 에이전트에서 사용할 수 있는 형식의 목록을 반환합니다. 반환된 목록에는 에이전트를 처리하는 데 적합한 Strands Agents MCPAgentTool 객체가 포함됩니다.

`tools/list` 호출은 페이지 단위로 이루어지므로, 함수는 `nextCursor` 필드가 더 이상 채워지지 않을 때까지 한 번에 한 페이지씩 도구를 가져오면서 반복해야 합니다. 이 유틸리티 함수는 HTTPS와 jsonrpc 프로토콜을 사용하여 엔드포인트를 직접 호출합니다. 이는 Strands Agents에서 제공하는 `MCPClient` 클래스보다 도구를 나열하는 저수준 방식입니다. 이 기능에 대해서는 나중에 살펴보겠습니다.

In [ ]:
def get_all_agent_tools_from_mcp_endpoint(gateway_endpoint, jwt_token, client):
    more_tools = True
    tools_count = 0
    tools_list = []

    requestBody = {"jsonrpc": "2.0", "id": 2, "method": "tools/list", "params": {}}
    next_cursor = ""

    while more_tools:
        if tools_count == 0:
            requestBody["params"] = {}
        else:
            print(f"\nGetting next page of tools since a next cursor was returned\n")
            requestBody["params"] = {"cursor": next_cursor}

        headers = {
            "Authorization": f"Bearer {jwt_token}",
            "Content-Type": "application/json",
        }

        print(f"\n\nListing tools for gateway {gateway_endpoint}")

        response = requests.post(gateway_endpoint, json=requestBody, headers=headers)

        tools_json = response.json()
        tools_count += len(tools_json["result"]["tools"])

        for tool in tools_json["result"]["tools"]:
            mcp_tool = MCPTool(
                name=tool["name"],
                description=tool["description"],
                inputSchema=tool["inputSchema"],
            )
            mcp_agent_tool = MCPAgentTool(mcp_tool, client)
            short_descr = tool["description"][0:40] + "..."
            print(f"adding tool '{mcp_agent_tool.tool_name}' - {short_descr}")
            tools_list.append(mcp_agent_tool)

        if "nextCursor" in tools_json["result"]:
            next_cursor = tools_json["result"]["nextCursor"]
            more_tools = True
        else:
            more_tools = False

    print(f"\nTotal tools found: {tools_count}\n")
    return tools_list

Lets use this helper function and see the results.

In [ ]:
client = MCPClient(
    lambda: streamablehttp_client(
        f"{gatewayEndpoint}", headers={"Authorization": f"Bearer {jwtToken}"}
    )
)
with client:
    all_tools = get_all_agent_tools_from_mcp_endpoint(
        gateway_endpoint=gatewayEndpoint, jwt_token=jwtToken, client=client
    )
    print(f"\nFound {len(all_tools)} tools using jsonrpc to list MCP tools\n")

#### Strands Agents list_tools_sync()를 페이지 매김과 함께 사용하기
Python 기반 MCP 클라이언트를 작성해 본 적이 있다면 클라이언트가 연결된 MCP 서버에서 사용 가능한 도구 세트를 반환하는 `list_tools_sync()` 메서드에 익숙할 것입니다.
그런데 MCP 목록 도구도 페이지 매김된다는 사실을 알고 계셨나요? 기본적으로 반환되는 도구의 첫 번째 작은
하위 세트만 받게 됩니다. 간단한 MCP 서버에서는 이 점을 알아차리지 못했을 수 있지만, 많은 실제 MCP 서버에서는 코드가 더 이상 페이지가 남지 않을 때까지 한 번에 여러 페이지의 도구를 가져오는 루프를 반복해야 합니다. 다음 유틸리티 `get_all_mcp_tools_from_mcp_client`는 바로 이러한 작업을 수행합니다.
이 유틸리티는 지정된 Strands Agent MCP 클라이언트에서 전체 도구 목록을 반환합니다.

In [ ]:
def get_all_mcp_tools_from_mcp_client(client):
    more_tools = True
    tools = []
    pagination_token = None
    while more_tools:
        tmp_tools = client.list_tools_sync(pagination_token=pagination_token)
        tools.extend(tmp_tools)
        if tmp_tools.pagination_token is None:
            more_tools = False
        else:
            more_tools = True
            pagination_token = tmp_tools.pagination_token
    return tools

Gateway를 사용하여 Python 클라이언트가 얼마나 많은 도구를 찾는지 확인해 보겠습니다.
먼저 엔드포인트 URL과 JWT 전달자 토큰을 기반으로 MCPClient 객체를 생성합니다. 그런 다음
MCP 서버에서 반환된 여러 페이지의 도구에 대한 전체 도구 세트를 검색합니다.
앞서 추가한 대상을 고려하면 300개 이상의 도구가 반환될 것입니다.

In [ ]:
client = MCPClient(
    lambda: streamablehttp_client(
        f"{gatewayEndpoint}", headers={"Authorization": f"Bearer {jwtToken}"}
    )
)
with client:
    all_tools = get_all_mcp_tools_from_mcp_client(client)
    print(f"\nFound {len(all_tools)} tools from list_tools_sync() on mcp client\n")

지금까지 Gateway를 MCP 서버로 사용하여 전체 도구 세트를 가져오는 세 가지 방법을 살펴보았습니다.

1. jsonrpc를 직접 사용하는 방법
2. Strands Agent MCPClient에서 `list_tools_sync()` 메서드를 사용하는 방법
3. MCP Inspector 도구(이후 jsonrpc를 사용)를 사용하는 방법

에이전트를 개발하는 일반적인 개발자는 옵션 2를 사용합니다.

### Gateway 내장 시맨틱 검색 도구 사용
이제 Gateway 내장 검색 도구를 사용하여 첫 번째 시맨틱 검색을 시도해 보겠습니다. 이 도구는 MCP 도구 목록에 추가되는 MCP 도구입니다.

먼저 MCP를 사용하여 검색 도구를 실행하는 간단한 유틸리티 함수를 정의해 보겠습니다.
도구 목록 작성과 마찬가지로 게이트웨이 엔드포인트와 JWT 토큰이 필요합니다. 그 외에는
검색 쿼리만 전달하면 됩니다. 게이트웨이 검색 도구가 나머지 작업을 처리합니다.
해당 쿼리를 자동으로 관리하는 서버리스 벡터 저장소와 비교합니다.

In [ ]:
def tool_search(gateway_endpoint, jwt_token, query):
    toolParams = {
        "name": "x_amz_bedrock_agentcore_search",
        "arguments": {"query": query},
    }
    toolResp = invoke_gateway_tool(
        gateway_endpoint=gateway_endpoint, jwt_token=jwt_token, tool_params=toolParams
    )
    tools = toolResp["result"]["structuredContent"]["tools"]
    return tools

In [ ]:
start_time = time.time()
tools_found = tool_search(
    gateway_endpoint=gatewayEndpoint,
    jwt_token=jwtToken,
    query="find me 3 credit research tools",
)
end_time = time.time()
print(
    f"tool search via direct Gateway invocation took {(end_time - start_time):.2f} seconds"
)
print(f"Top tool: {tools_found[0]['name']}")

대부분의 경우 1초 이내에 검색 결과가 반환되는 속도에 주목하세요. 결과는
검색어와 도구 메타데이터의 일치 여부를 기준으로 검색 관련성 순으로 내림차순으로 반환됩니다.
가장 관련성 높은 도구가 목록의 맨 위에 표시됩니다. 검색을 처음 구현하면 최대
10개의 결과가 반환됩니다. 그런 다음 에이전트에서 이러한 모든 도구를 사용하거나 가장 관련성 높은 일치 항목 중 일부만 선택할 수 있습니다.

# 많은 도구를 가진 MCP 서버와 함께 Strands Agents 사용

먼저, Strands Agent와 함께 사용할 모델을 선택합니다.
이 노트북에서는 Amazon Bedrock 모델을 사용하지만, Strands와 AgentCore는
모든 LLM과 호환됩니다.

In [ ]:
bedrockmodel = BedrockModel(
    model_id="us.anthropic.claude-3-7-sonnet-20250219-v1:0",
    temperature=0.7,
    streaming=True,
    boto_session=session,
)

#### AgentCore Gateway를 사용한 간단한 Strands 에이전트(에이전트 도구)
이제 AgentCore Gateway에서 제공하는 MCP 서버를 Strands 에이전트를 사용하여 활용하는 것이 얼마나 쉬운지 살펴보겠습니다.
이 간단한 예시에서는 에이전트에게 숫자를 더하도록 요청합니다.

In [ ]:
jwtToken = utils.get_bearer_token(
    client_id=cognito_response["client_id"],
    username="testuser",
    password="MyPassword123!",
)
client = MCPClient(
    lambda: streamablehttp_client(
        f"{gatewayEndpoint}", headers={"Authorization": f"Bearer {jwtToken}"}
    )
)
with client:
    all_tools = get_all_mcp_tools_from_mcp_client(client)
    print(f"\nFound {len(all_tools)} tools from list_tools_sync() on mcp client\n")

    simple_agent = Agent(
        model=bedrockmodel, tools=all_tools, callback_handler=null_callback_handler
    )
    result = simple_agent("add 100 plus 50 pass ")
    print(f"{result.message['content'][0]['text']}")

Strands Agents 프레임워크를 사용하면 에이전트 이벤트 루프를 우회하여 MCP 도구를 직접 호출할 수도 있습니다.
Gateway 도구는 네이티브 MCP 도구로 제공되므로 Gateway 도구에 대해서도 이 작업을 수행할 수 있습니다. 여기서는 `agent.tool.<tool_name>(args)` 구문을 사용하여 Gateway MCP 도구를 호출합니다.

```python
direct_result = simple_agent.tool.Calc2___add_numbers(firstNumber=10, secondNumber=20)
resp_json = json.loads(direct_result['content'][0]['text'])
```

In [ ]:
jwtToken = utils.get_bearer_token(
    client_id=cognito_response["client_id"],
    username="testuser",
    password="MyPassword123!",
)
client = MCPClient(
    lambda: streamablehttp_client(
        f"{gatewayEndpoint}", headers={"Authorization": f"Bearer {jwtToken}"}
    )
)
with client:
    all_tools = get_all_mcp_tools_from_mcp_client(client)
    print(f"\nFound {len(all_tools)} tools from list_tools_sync() on mcp client\n")

    simple_agent = Agent(
        model=bedrockmodel, tools=all_tools, callback_handler=null_callback_handler
    )
    direct_result = simple_agent.tool.Calc2___add_numbers(
        firstNumber=10, secondNumber=20
    )
    print(f"direct result = {direct_result}")

In [ ]:
def get_search_tool(client):
    mcp_tool = MCPTool(
        name="x_amz_bedrock_agentcore_search",
        description="A special tool that returns a trimmed down list of tools given a context. Use this tool only when there are many tools available and you want to get a subset that matches the provided context.",
        inputSchema={
            "type": "object",
            "properties": {
                "query": {
                    "type": "string",
                    "description": "search query to use for finding tools",
                }
            },
            "required": ["query"],
        },
    )
    return MCPAgentTool(mcp_tool, client)

In [ ]:
def search_using_strands(client, query):
    simple_agent = Agent(
        model=bedrockmodel,
        tools=[get_search_tool(client)],
        callback_handler=null_callback_handler,
    )

    direct_result = simple_agent.tool.x_amz_bedrock_agentcore_search(query=query)

    resp_json = json.loads(direct_result["content"][0]["text"])
    search_results = resp_json["tools"]
    # print(json.dumps(search_results, indent=4))
    return search_results

In [ ]:
def find_strands_tools(client, query, top_n):
    strands_mcp_tools = []
    results = search_using_strands(client, query)
    for tool in results[:top_n]:
        mcp_tool = MCPTool(
            name=tool["name"],
            description=tool["description"],
            inputSchema=tool["inputSchema"],
        )
        strands_mcp_tools.append(MCPAgentTool(mcp_tool, client))
    return strands_mcp_tools

In [ ]:
jwtToken = utils.get_bearer_token(
    client_id=cognito_response["client_id"],
    username="testuser",
    password="MyPassword123!",
)
client = MCPClient(
    lambda: streamablehttp_client(
        f"{gatewayEndpoint}", headers={"Authorization": f"Bearer {jwtToken}"}
    )
)
with client:
    simple_agent = Agent(
        model=bedrockmodel,
        tools=[get_search_tool(client)],
        callback_handler=null_callback_handler,
    )

    direct_result = simple_agent.tool.x_amz_bedrock_agentcore_search(
        query="find equity trading tools"
    )

    resp_json = json.loads(direct_result["content"][0]["text"])
    search_results = resp_json["tools"]
    print(json.dumps(search_results, indent=4))

In [ ]:
jwtToken = utils.get_bearer_token(
    client_id=cognito_response["client_id"],
    username="testuser",
    password="MyPassword123!",
)
client = MCPClient(
    lambda: streamablehttp_client(
        f"{gatewayEndpoint}", headers={"Authorization": f"Bearer {jwtToken}"}
    )
)
with client:
    results = search_using_strands(client, "find trading tools")
    print(json.dumps(search_results[0], indent=4))

    results = search_using_strands(client, "find credit research tools")
    print(json.dumps(search_results[0], indent=4))

# Strands Agent에 도구 검색 결과 추가

이제 검색에서 반환된 도구를 Strands 에이전트에 추가하는 방법을 살펴보겠습니다. 코딩을 간소화하기 위해 도구 검색 결과를 Strands MCPAgentTool 객체에 매핑하는 유틸리티 함수를 제공하겠습니다. 검색 결과를 전달하고, 에이전트에 전달할 결과 수를 지정하기만 하면 됩니다.

In [ ]:
def tools_to_strands_mcp_tools(tools, top_n):
    strands_mcp_tools = []
    for tool in tools[:top_n]:
        mcp_tool = MCPTool(
            name=tool["name"],
            description=tool["description"],
            inputSchema=tool["inputSchema"],
        )
        strands_mcp_tools.append(MCPAgentTool(mcp_tool, client))
    return strands_mcp_tools

In [ ]:
jwtToken = utils.get_bearer_token(
    client_id=cognito_response["client_id"],
    username="testuser",
    password="MyPassword123!",
)
client = MCPClient(
    lambda: streamablehttp_client(
        f"{gatewayEndpoint}", headers={"Authorization": f"Bearer {jwtToken}"}
    )
)
with client:
    agent = Agent(
        model=bedrockmodel,
        tools=find_strands_tools(
            client,
            "tools for doing addition, subtraction, multiplication, division",
            10,
        ),
    )
    result = agent("(10*2)/(5-3)")
    print(f"{result.message['content'][0]['text']}")

In [ ]:
%%time

jwtToken = utils.get_bearer_token(
    client_id=cognito_response["client_id"],
    username="testuser",
    password="MyPassword123!",
)
client = MCPClient(
    lambda: streamablehttp_client(
        f"{gatewayEndpoint}", headers={"Authorization": f"Bearer {jwtToken}"}
    )
)
with client:
    print("Searching for an ADDING tool from endpoint with full set of tools...")
    tools_found = tool_search(
        gateway_endpoint=gatewayEndpoint,
        jwt_token=jwtToken,
        query="tools for multiplying two numbers",
    )
    print(f"Top tool found: {tools_found[0]['name']}\n")

    agent = Agent(model=bedrockmodel, tools=tools_to_strands_mcp_tools(tools_found, 1))
    result = agent("10 * 70")
    print(f"{result.message['content'][0]['text']}")

지연 시간 개선에 주목하세요. Gateway 검색 도구의 하위 집합을 사용한 이 예시는 수백 개의 도구에 의존하는 에이전트 호출보다 훨씬 빠릅니다.

# 도구 검색을 사용하여 3배 지연 시간 개선 보여주기

이제 Strands 에이전트에서 Gateway MCP 도구를 사용하는 방법과 도구를 검색하여 에이전트에 추가하는 방법을 알았으니, 검색의 힘을 보여드리겠습니다. 상당한 지연 시간 단축과 입력 토큰 사용량을 강조하여 보여드리겠습니다.

지연 시간과 토큰 사용량 감소 효과를 보여주기 위해 두 가지 접근 방식을 나란히 비교해 보겠습니다.

1. **검색 사용 안 함**: MCP 서버가 제공하는 전체 MCP 도구 세트(이 경우 300개 이상)를 에이전트에 추가하고, 에이전트가 도구를 선택하고 호출하도록 합니다.
2. **검색 사용**: 두 번째 접근 방식에서는 해당 주제를 기반으로 검색을 수행하고 가장 관련성 높은 도구만 에이전트에 전송합니다. 이를 증명하기 위해 수학(숫자 덧셈)과 음식(식당 예약)이라는 두 가지 주제를 사용하며, 각 주제에는 서로 다른 도구 세트가 필요합니다.

지연 시간 분포를 정규화하고 의미 있는 비교를 얻기 위해 각 접근 방식을 여러 번 반복합니다.
또한, 검색 방식을 적용할 때 이점을 과장하지 않기 위해 에이전트 호출 지연 시간뿐만 아니라 도구 검색 수행 지연 시간도 포함합니다. 각 반복마다 에이전트에 두 가지 작업을 할당합니다.

1. 수학 작업 - 숫자 두 개 더하기
2. 음식 작업 - 식당 예약

아래 결과는 지연 시간이 3배 감소하고 입력 토큰 사용량이 훨씬 더 크게 감소하는 이점을 보여줍니다.
토큰 사용량 절감은 비용 절감으로 이어지지만, 입력 토큰 비용이 상대적으로 낮기 때문에 그 효과는 크지 않을 수 있습니다(많은 모델 제공업체에서 입력 토큰은 훨씬 저렴합니다). 그러나 대규모 에이전트 배포의 경우 입력 토큰 사용 비용도 누적될 수 있으므로 동적 검색을 통해 에이전트 런타임 비용도 줄일 수 있습니다.

#### MCP 도구 전체를 사용하여 에이전트의 대기 시간과 토큰 사용량을 측정합니다.

In [ ]:
iterations = 2
full_tokens = light_tokens = 0
full_elapsed_time = light_elapsed_time = 0

jwtToken = utils.get_bearer_token(
    client_id=cognito_response["client_id"],
    username="testuser",
    password="MyPassword123!",
)
client = MCPClient(
    lambda: streamablehttp_client(
        f"{gatewayEndpoint}", headers={"Authorization": f"Bearer {jwtToken}"}
    )
)

In [ ]:
with client:
    all_tools = get_all_mcp_tools_from_mcp_client(client)
    print(f"\nFound {len(all_tools)} tools from list_tools_sync() on mcp client\n")
    heavy_agent = Agent(
        model=bedrockmodel, tools=all_tools, callback_handler=null_callback_handler
    )

    math_input = "add 100 plus <iteration>"
    food_input = (
        "book me a table for 2 at Burger King under name Jo Smith at 7pm August <day>"
    )

    print("using agent with ALL tools...")
    start_time = time.time()

    for i in range(iterations):
        result = heavy_agent(math_input.replace("<iteration>", str(i + 1)))
        print(f"{i+1}) {result.message['content'][0]['text']}")

        result = heavy_agent(food_input.replace("<day>", str(i + 1)))
        print(f"{i+1}) {result.message['content'][0]['text']}")

    end_time = time.time()
    full_tokens = result.metrics.accumulated_usage["totalTokens"]
    full_elapsed_time = end_time - start_time
    print(f"\nTotal time: {full_elapsed_time:.1f} s, tokens: {full_tokens:,d}\n")

#### 게이트웨이 검색을 사용하여 상담원의 지연 시간 및 토큰 사용량 측정
이제 동적 접근 방식을 사용하여 검색을 호출하여 관련 도구를 찾은 다음, 해당 도구만 사용하여 상담원에게 전화를 겁니다. 각 대화 턴마다 상담원을 재설정하므로, 이전 턴의 대화 기록에서 메시지 목록도 초기화됩니다.

In [ ]:
with client:
    print("using agent with ONLY tools from focused search...")
    start_time = time.time()
    messages = []

    light_agent = Agent()

    for i in range(iterations):
        print("Searching for an ADDING tool from endpoint with full set of tools...")
        tools_found = tool_search(
            gateway_endpoint=gatewayEndpoint,
            jwt_token=jwtToken,
            query="tools for simply adding two numbers",
        )
        print(f"Top tool found: {tools_found[0]['name']}\n")
        light_agent = Agent(
            model=bedrockmodel,
            tools=tools_to_strands_mcp_tools(tools_found, 1),
            messages=messages,
            callback_handler=null_callback_handler,
        )
        light_result = light_agent(math_input.replace("<iteration>", str(i + 1)))
        print(f"{i+1}) {light_result.message['content'][0]['text']}")
        messages = light_agent.messages

        print(
            "Searching for a RESTAURANT BOOKING tool from endpoint with full set of tools..."
        )
        tools_found = tool_search(
            gateway_endpoint=gatewayEndpoint,
            jwt_token=jwtToken,
            query="tools for booking a restaurant reservation",
        )
        print(f"Top tool found: {tools_found[0]['name']}\n")
        light_agent = Agent(
            model=bedrockmodel,
            tools=tools_to_strands_mcp_tools(tools_found, 1),
            messages=messages,
            callback_handler=null_callback_handler,
        )
        light_result = light_agent(food_input.replace("<day>", str(i + 1)))
        print(f"{i+1}) {light_result.message['content'][0]['text']}")
        messages = light_agent.messages
        light_tokens = light_result.metrics.accumulated_usage["totalTokens"]
    end_time = time.time()

    light_elapsed_time = end_time - start_time
    print(f"\nTotal time: {light_elapsed_time:.1f} s, tokens: {light_tokens:,d}\n")

#### Compare results, higlighting benefits of search

In [ ]:
print(
    f"\n\nLatency without search: {full_elapsed_time:.1f}s, using search: {light_elapsed_time:.1f}s"
)
print(f"Tokens without search: {full_tokens:,d}, using search: {light_tokens:,d}")

# 결론
이 튜토리얼에서는 Amazon Bedrock AgentCore Gateway와 내장된 완전 관리형 시맨틱 검색 기능에 대해 알아보았습니다. 다음 내용을 살펴보았습니다.

- 시맨틱 검색이 활성화된 게이트웨이를 생성하는 방법
- 단일 엔드포인트에서 300개 이상의 MCP 도구를 표시하기 위해 여러 게이트웨이 대상을 추가하는 방법
- 3가지 접근 방식을 사용하여 게이트웨이에 있는 도구를 나열하는 방법
- 내장된 시맨틱 검색 도구를 사용하여 관련 도구를 찾는 방법
- 검색 기능을 Strands Agent와 통합하는 방법
- 수백 개의 도구가 있는 서버를 사용하는 에이전트와 시맨틱 검색을 사용하여 도구 범위를 특정 주제로 좁히는 에이전트의 성능을 비교하는 방법

AgentCore Gateway 검색은 고급 사용 사례에도 유용합니다. 검색 기능을 단순한 제어 플레인 API가 아닌 기본 MCP 도구로 제공함으로써 에이전트가 새로운 MCP 서버를 검색하고 런타임에 새로운 기능을 찾을 수 있도록 더 많은 자율성을 부여하여 더 어려운 문제를 해결하는 데 획기적인 진전을 이룰 수 있습니다.
또한 검색 기능은 MCP 레지스트리와 지원 에이전트 개발자가 새로운 에이전트를 설계하고 구축할 때 중요한 기반이 됩니다.

# 리소스 정리

먼저 AgentCore Gateway 리소스를 정리하기 위한 몇 가지 도우미 함수를 정의해 보겠습니다.

In [ ]:
def delete_gatewaytarget(gateway_id):
    response = agentcore_client.list_gateway_targets(gatewayIdentifier=gateway_id)

    print(f"Found {len(response['items'])} targets for the gateway")

    for target in response["items"]:
        print(
            f"Deleting target with Name: {target['name']} and Id: {target['targetId']}"
        )

        response = agentcore_client.delete_gateway_target(
            gatewayIdentifier=gateway_id, targetId=target["targetId"]
        )
        time.sleep(20)


def delete_gateway(gateway_id):
    response = agentcore_client.delete_gateway(gatewayIdentifier=gateway_id)

### Deleting Gateway Targets

In [ ]:
delete_gatewaytarget(gateway_id=gatewayId)

### Deleting the Gateway itself

In [ ]:
delete_gateway(gateway_id=gatewayId)

In [ ]:
lambda_arns = [
    calc_lambda_resp["lambda_function_arn"],
    restaurant_lambda_resp["lambda_function_arn"],
]

for arn in lambda_arns:
    if utils.delete_gateway_lambda(arn):
        print(f"Deleted Lambda: {arn}")
    else:
        print(f"Lambda {arn} not found or deletion failed")

In [ ]:
# Gateway role cleanup
if utils.delete_gateway_iam_role():
    print("Gateway IAM role deleted")
else:
    print("Gateway IAM role not found or deletion failed")

# Cognito cleanup
if utils.delete_cognito_user_pool():
    print("Cognito pool deleted")
else:
    print("✗ Failed to delete Cognito pool")